In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import Image, display
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import pylab as pl
import re
import codecs
import nltk
import pymorphy2
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

%pylab inline
pylab.rcParams['figure.figsize'] = (15,10)

Populating the interactive namespace from numpy and matplotlib


---
Устанавливаю необходимые библиотеки


In [3]:
conda config --append channels conda-forge


Note: you may need to restart the kernel to use updated packages.


In [4]:
conda install pymorphy2

Solving environment: done

## Package Plan ##

  environment location: /Users/olgakalinina/opt/anaconda3

  added / updated specs:
    - pymorphy2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.12.0               |   py39hecd8cb5_0        14.5 MB
    dawg-python-0.7.2          |     pyhd8ed1ab_0          12 KB  conda-forge
    docopt-0.6.2               |   py39hecd8cb5_0          24 KB
    pymorphy2-0.9.1            |     pyhd8ed1ab_0          44 KB  conda-forge
    pymorphy2-dicts-ru-2.4.417127.4579844|     pyhd8ed1ab_0         7.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        21.6 MB

The following NEW packages will be INSTALLED:

  dawg-python        conda-forge/noarch::dawg-python-0.7.2-pyhd8ed1ab_0
  docopt             pkgs/main/osx-64::docopt-0.6.2-py39hecd8cb5_0
  pymorphy2     

In [9]:
 pip install --upgrade pip -m /Users/olgakalinina/opt/anaconda3/lib/python3.9 


Usage:   
  /Users/olgakalinina/opt/anaconda3/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /Users/olgakalinina/opt/anaconda3/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /Users/olgakalinina/opt/anaconda3/bin/python -m pip install [options] [-e] <vcs project url> ...
  /Users/olgakalinina/opt/anaconda3/bin/python -m pip install [options] [-e] <local project path> ...
  /Users/olgakalinina/opt/anaconda3/bin/python -m pip install [options] <archive url/path> ...

no such option: -m
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


---

## Семантический анализ твитов

Сегодня мы построим классификатор, который будет разделять текст на позитивные и негативные высказывания. Для этого мы воспользуемся уже размеченной базой.
Загрузим данные для анализа

In [3]:
df = pd.read_excel('../data-samples/tweets_example.xlsx')
df.loc[16:25]

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
16,408906854803451904,1386325965,fantanshik,"RT @Abdullin_A_R: @LikhodedovaMary эхх, а в УГ...",1,0,1,0,2074,82,44,5
17,408906854849204224,1386325965,Tienn_En,"@marinaysol а, а то подумала, что у тебя там п...",1,0,0,0,6362,30,28,1
18,408906855146983424,1386325966,bstrd666,@xLesherx @4EU3 зря вы с этой хуйней шутите)) ...,1,0,0,0,13431,473,111,2
19,408906857659392000,1386325966,esken_h,@Moscow_advokat Очень главное спасибо for МЕ...,1,0,0,0,126795,581,86,10
20,408906857982726080,1386325966,Obrazcova98,"У нас есть прекрасная история, как сдохнуть за...",1,0,0,0,492,14,23,0
21,408906762813579328,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
22,408906818262687680,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
23,408906858515398720,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
24,408906914437685184,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
25,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


---

Все колонки таблицы могут содержать информацию о тональности твита, но мы будем ориентироваться исключительно на текст и на столбец отнесения к классу positiv.
Заменим значение -1 в колонке positive на 0

In [4]:
df.positive[df.positive==-1]=0
df.loc[16:25]

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
16,408906854803451904,1386325965,fantanshik,"RT @Abdullin_A_R: @LikhodedovaMary эхх, а в УГ...",1,0,1,0,2074,82,44,5
17,408906854849204224,1386325965,Tienn_En,"@marinaysol а, а то подумала, что у тебя там п...",1,0,0,0,6362,30,28,1
18,408906855146983424,1386325966,bstrd666,@xLesherx @4EU3 зря вы с этой хуйней шутите)) ...,1,0,0,0,13431,473,111,2
19,408906857659392000,1386325966,esken_h,@Moscow_advokat Очень главное спасибо for МЕ...,1,0,0,0,126795,581,86,10
20,408906857982726080,1386325966,Obrazcova98,"У нас есть прекрасная история, как сдохнуть за...",1,0,0,0,492,14,23,0
21,408906762813579328,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,0,0,0,0,8064,111,94,2
22,408906818262687680,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",0,0,0,0,26,42,39,0
23,408906858515398720,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,0,0,0,0,718,49,249,0
24,408906914437685184,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",0,0,0,0,10628,207,200,0
25,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",0,0,0,0,35,17,34,0


### Очистка и предобработка данных

Перед разработкой классификатора нам необходимо очистить и предобработать данные.

Начнем с очистки данных

----------------------------

***Внимание!*** Библиотека [*nltk*](https://www.nltk.org) может содержать не все компоненты. В случае возникновения ошибки необходимо запустить скрипт

*import nltk   
nltk.download()*

В открывшемся окне необходимо выбрать и установить требуемые компоненты

----------------------------

In [69]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Загрузим БД с позитивными твитами

In [5]:
pos=pd.read_csv('../datasets/positive.csv', sep=';', header=None)

Загрузим БД с негативными твитами

In [6]:
neg=pd.read_csv('../datasets/negative.csv', sep=';', header=None)

Объединим БД

In [7]:
result=pd.concat([pos, neg], ignore_index=True)

Переименуем название столбцов

In [8]:
result=result.rename(columns={0 : 'id', 1 : 'date', 2: 'name', 3: 'text', 4: 'positive', 5: 'rep', 6: 'rtv', 7: 'fav', 8: 'total_count', 9: 'fol', 10: 'friends', 11: 'list_count'})

Посмотрим, сколько всего твитов, сколько из них - негативных, сколько - позитивных

In [9]:
contents= {'twits': result.id.count(), 'positive': result.query("positive==1").agg("positive").count(), 'negative': result.query("positive==-1").agg("positive").count()}

In [10]:
pd.DataFrame(contents, index=[0])

,twits,positive,negative
0,226834,114911,111923


Если твит позитивный, то значение колонки positive = 0

In [12]:
result.positive[result.positive==-1]=0

Произведем очистку данных

----

***Совет:*** сохраняйте промежуточные результаты очистки, чтобы в случае неверных действий на каком-либо этапе не пересчитывать все предыдущие этапы

---

Заменяю заглавные буквы на строчные

In [13]:
result.text = result.text.str.lower()
result.text.loc[19:22]

19    @moscow_advokat очень главное спасибо for   ме...
20    у нас есть прекрасная история, как сдохнуть за...
21    @benjamin1610 сегодня столько поводов выпить, ...
22    @stalingulag унылый?...наверное...вдруг его пе...
Name: text, dtype: object

Уберу латинские буквы и оставлю только кириллицу

In [14]:
result.text = result.text.str.replace(r"[^А-Яа-я]"," ")
result.text.loc[19:22]

19                    очень главное спасибо       ме...
20    у нас есть прекрасная история  как сдохнуть за...
21                  сегодня столько поводов выпить  ...
22                 унылый    наверное   вдруг его пе...
Name: text, dtype: object

Один твит разделю на несколько слов

In [15]:
from nltk.tokenize import word_tokenize
result.text = list(map(word_tokenize, result.text))
result.text.loc[19:22]

19    [очень, главное, спасибо, медвед, он, работал,...
20    [у, нас, есть, прекрасная, история, как, сдохн...
21    [сегодня, столько, поводов, выпить, что, я, уж...
22    [унылый, наверное, вдруг, его, перед, этим, чп...
Name: text, dtype: object

Найду стоп-слова - слова, которые не несут смысловой нагрузки

In [16]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()
russian_stopwords

['а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

Удалю стоп-слова

In [17]:
def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

result.text = list(map(delete_stopword, result.text))
result.text.loc[19:22]

19    [очень, главное, спасибо, медвед, работал, кло...
20              [прекрасная, история, сдохнуть, неделю]
21    [сегодня, столько, поводов, выпить, могу, усид...
22                   [унылый, наверное, этим, чпокнули]
Name: text, dtype: object

Проведу лемматизацию, то есть приведу слова к начальной форме

In [18]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s

result.text = list(map(lemmatization, result.text))
result.text.loc[19:22]

19    [очень, главное, спасибо, медведа, работать, к...
20              [прекрасный, история, сдохнуть, неделя]
21    [сегодня, столько, повод, выпить, мочь, усидет...
22                    [унылый, наверное, это, чпокнуль]
Name: text, dtype: object

Удаляю слова, которые встречаются один раз, так как они не смогут помочь предсказать позитивный или негативный твит

In [19]:
from nltk.probability import FreqDist

def to_str(s):
    new_s = ' '.join(j for j in s)
    return new_s

text_tokens = word_tokenize(' '.join(j for j in list(map(to_str, result.text))))
text = nltk.Text(text_tokens)
fdist = FreqDist(text)
words_to_del = list(filter(lambda k: fdist[k] == 1, fdist))

def delete_word(words):
    global words_to_del
    new_s = [word for word in words if word not in words_to_del]
    return new_s

result.text = list(map(delete_word, result.text))
result.text = list(map(to_str, result.text))
result.text.loc[19:22]

19    очень главное спасибо медведа работать клоун а...
20                   прекрасный история сдохнуть неделя
21      сегодня столько повод выпить мочь усидеть место
22                                  унылый наверное это
Name: text, dtype: object

Найду количество слов до очистки, количество уникальных слов, количество слов после очистки

In [20]:
len(text_tokens), len(words_to_del), len(text_tokens) - len(words_to_del)

(1493275, 53568, 1439707)

Найду количество пустых твитов, из них позитивные -

In [21]:
len(result[result.text == '']), len(result[(result.text == '') & (result.positive == 1)])

(856, 382)

Удалю пустые твиты и посмотрю, сколько всего осталось твитов

In [22]:
result = result.drop(result[result.text == ''].index, axis = 0)
len(result)

225978

---

Кодирую данные методом мешка слов и TF-IDF, построю классификатор на осонове логистической регресии и случайного леса и сравню результаты

---

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(result.text, result.positive, test_size=0.3, random_state=21)

Кодирую данные с помощью мешка слов и tf-idf

In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
cv_train = cv.fit_transform(X_train)
cv_test = cv.transform(X_test)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
tfidf_test = tfidf.transform(X_test)

Построю классификатор, используя логистическую регрессию и кодировку методом мешок слов

In [25]:
lr = LogisticRegression(random_state=21)
lr.fit(cv_train, y_train)
cv_pred = lr.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, lr.predict(cv_train)))

test
              precision    recall  f1-score   support

           0       0.72      0.71      0.71     33422
           1       0.72      0.73      0.72     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

           0       0.80      0.79      0.79     78027
           1       0.80      0.80      0.80     80157

    accuracy                           0.80    158184
   macro avg       0.80      0.80      0.80    158184
weighted avg       0.80      0.80      0.80    158184



Построю классификатор, используя логистическую регрессию и кодировку методом TF_IDF

In [27]:
lr = LogisticRegression(random_state=21)
lr.fit(tfidf_train, y_train)
tfidf_pred = lr.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, lr.predict(tfidf_train)))

test
              precision    recall  f1-score   support

           0       0.73      0.69      0.71     33422
           1       0.71      0.75      0.73     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

           0       0.79      0.75      0.77     78027
           1       0.77      0.80      0.79     80157

    accuracy                           0.78    158184
   macro avg       0.78      0.78      0.78    158184
weighted avg       0.78      0.78      0.78    158184



Построю классификатор, используя случайный лес и кодировку методом мешок слов

In [28]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=250, n_jobs=-1, random_state=21)
forest.fit(cv_train, y_train)
cv_pred = forest.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, forest.predict(cv_train)))

test
              precision    recall  f1-score   support

           0       0.68      0.76      0.72     33422
           1       0.74      0.65      0.69     34372

    accuracy                           0.70     67794
   macro avg       0.71      0.70      0.70     67794
weighted avg       0.71      0.70      0.70     67794

train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     78027
           1       1.00      1.00      1.00     80157

    accuracy                           1.00    158184
   macro avg       1.00      1.00      1.00    158184
weighted avg       1.00      1.00      1.00    158184



Построю классификатор, используя случайный лес и кодировку методом TF_IDF

In [29]:
forest.fit(tfidf_train, y_train)
tfidf_pred = forest.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, forest.predict(tfidf_train)))

test
              precision    recall  f1-score   support

           0       0.72      0.70      0.71     33422
           1       0.72      0.74      0.73     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     78027
           1       1.00      1.00      1.00     80157

    accuracy                           1.00    158184
   macro avg       1.00      1.00      1.00    158184
weighted avg       1.00      1.00      1.00    158184



В результате получили, что, несмотря на использованный метод и кодировку, результаты получились одинаковыми.